# Introduction

We are going to implement a very simple agent-based model of disease spread (an SI - susceptible-infected model) with some basic visualisation.

# Prelude

We need to tell the system about some code we are going to use. `using` is Julia's `library`, `include` works the same way as `source`.

In [ ]:
using Random
using Luxor

include("SimpleAgentEvents/src/SimpleAgentEvents.jl")

# these modules are declared in the file we just included
using .SimpleAgentEvents
using .SimpleAgentEvents.Scheduler

# Type declarations

Now we need to tell Julia about how the objects look like that we are going to use. In contrast to R, most objects you want to use in Julia have a fixed layout that does not change (PS: R is the outlier here, most languages have structs, classes, records or similar).

An `@enum` is a type that can only have a handful of named values, e.g.:
```Julia
@enum Boolean true false
```

A `struct` is like a list in R, but with a fixed number of elements (ideally with a fixed type).

In [ ]:
# all possible states a person can be in
@enum Status susceptible infected

# this is our agent type
mutable struct Person
    # state
    status :: Status
    # other agents this one can infect or be infected by
    contacts :: Vector{Person}
    # location in the world
    x :: Float64
    y :: Float64
end

# how we construct a person object
Person(x, y) = Person(susceptible, [], x, y)
Person(state, x, y) = Person(state, [], x, y)

The simulation itself is mainly used for bookkeeping.

In [ ]:
mutable struct Simulation
    # the scheduler keeps track of which agent is going to do what when
    scheduler :: PQScheduler{Float64}
    # parameters:
    # infection rate
    inf :: Float64
    # recovery rate
    rec :: Float64

    # and this is our population of agents
    pop :: Vector{Person}
end

# the simulation system needs this for technical reasons
scheduler(sim :: Simulation) = sim.scheduler

Simulation(i, r) = Simulation(PQScheduler{Float64}(), i, r, [])

# The model

Now we can write our actual model. I am using a set of macros (declared in the SimpleAgentEvents file we included further up) that make it possible to create simple event-based simulations in a relatively straightforward manner.

In [ ]:
# usage:
# processes NAME SIMULATION AGENT ACTIONS
@processes SI sim person::Person begin
 
    # each action has the format
    # distribution(rate) ~ condition => action
    # currently only poisson is implemented
    
    # infection with rate proportional to #infected contacts
    @poisson(sim.inf * count(p -> p.status == infected, person.contacts)) ~
        # a person has to be susceptible in order to get infected
        person.status == susceptible        => 
            begin
                # status changes
                person.status = infected
                # *important*: 
                # return all agents that need to be activated
                # including *all* whose circumstances have changed
                [person; person.contacts]
            end

    # recovery
    @poisson(sim.rec)  ~
        person.status == infected           => 
            begin
                person.status = susceptible
                person.contacts
            end
end

# and this is the entire model

# Running the model

Now we can create the simulation and run it. The population setup is somewhat fiddly at this stage, so I have put it into a separate file.

In [ ]:
include("setup_world.jl")

This sets up the simulation and the agents.

In [ ]:
# create a simulation object with parameter values
sim = Simulation(0.5, 0.1)
# create a population of agents living on a 
# 50x50 grid (von Neumann)
sim.pop = setup_grid(50, 50)
# top left agent is infected
sim.pop[1].status = infected

# spawn activates agents
# this function was generated by the model declaration
for person in sim.pop
    spawn_SI(person, sim)
end

# for reproducibility
Random.seed!(42);

And this runs the simulation for a few time steps.

In [ ]:
for t in  1:10
    # tell the scheduler to run all actions up to t+1
    upto!(sim.scheduler, time_now(sim.scheduler) + 1.0)
    # a bit of output
    println(time_now(sim.scheduler), ", ", 
        count(p -> p.status == infected, sim.pop), ", ",
        count(p -> p.status == susceptible, sim.pop))
end

# Visualisation

It is generally a good idea to have a proper visualisation for a simulation. It makes it a lot easier to catch bugs, over time it provides an intuition of the system's behaviour and it is fun.

Usually it would be good to have a dynamic display of the system state over time, but for this simple example we will make do with static output. If you want to track the progress of the simulation you can just execute the previous block again and then rerun the visualisation.

In [ ]:
# this uses the Luxor package, which allows for very straighforward 
# generation of SVG drawings
@svg begin
    # scale from model coordinates to screen coordinates
    f_scale = 11
    # size of agents
    p_size = 5
    
    # otherwise we'll only have to lower right quarter
    # (don't ask...)
    origin(0, 0)

    # paint it black
    sethue("black")
    
    # ** first draw all connections
    # (we don't want to worry about agent size here,
    #  so we just draw the lines from x1,y1 to x2,y2
    #  and paint the agents on top of that)
    
    # for each agent
    for p in sim.pop
        x1 = p.x * f_scale
        y1 = p.y * f_scale
        
        # draw all lines to other agents
        for p2 in p.contacts
            x2 = p2.x * f_scale
            y2 = p2.y * f_scale
            
            setline(1)
            setdash("solid")
            line(Point(x1, y1), Point(x2, y2), :stroke)
        end
    end
    # yes, this draws all lines twice
    # who cares

    # ** now draw all agents
    
    for p in sim.pop
        x = p.x * f_scale
        y = p.y * f_scale
        
        # show the status as colours
        if p.status == infected
            sethue("red")
        elseif p.status == susceptible
            sethue("blue")
        end
        
        circle(Point(x, y), p_size, :fill)
    end
end

# Things to try

* Run for longer, is there an equilibrium or an end state?
* Try different values for `inf` and `rec`.
* Does changing the shape of the world (`x` vs. `y` size) have an effect?
* Add more patient zeros, e.g. at 50, 50.